In [10]:
import kagglehub
import zipfile
import os

# Download latest version
dataset_path = kagglehub.dataset_download("mostafaabla/garbage-classification")
print("Downloaded dataset path:", dataset_path)

# Extract if it's a zip
if dataset_path.endswith(".zip"):
    with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
        zip_ref.extractall("garbage_dataset")
    dataset_dir = "garbage_dataset"
else:
    dataset_dir = dataset_path

print("Dataset extracted to:", dataset_dir)


Downloaded dataset path: /Users/rodynaamr/.cache/kagglehub/datasets/mostafaabla/garbage-classification/versions/1
Dataset extracted to: /Users/rodynaamr/.cache/kagglehub/datasets/mostafaabla/garbage-classification/versions/1


In [13]:
import os

dataset_dir = "/Users/rodynaamr/.cache/kagglehub/datasets/mostafaabla/garbage-classification/versions/1/garbage_classification"

image_paths = []
labels = []

for cls in os.listdir(dataset_dir):
    cls_path = os.path.join(dataset_dir, cls)
    if os.path.isdir(cls_path):
        for img_file in os.listdir(cls_path):
            if img_file.lower().endswith((".png", ".jpg", ".jpeg")):
                image_paths.append(os.path.join(cls_path, img_file))
                labels.append(cls)

print(f"Total images found: {len(image_paths)}")
print("First 5 samples:")
for i in range(5):
    print(image_paths[i], "->", labels[i])


Total images found: 15515
First 5 samples:
/Users/rodynaamr/.cache/kagglehub/datasets/mostafaabla/garbage-classification/versions/1/garbage_classification/paper/paper903.jpg -> paper
/Users/rodynaamr/.cache/kagglehub/datasets/mostafaabla/garbage-classification/versions/1/garbage_classification/paper/paper917.jpg -> paper
/Users/rodynaamr/.cache/kagglehub/datasets/mostafaabla/garbage-classification/versions/1/garbage_classification/paper/paper724.jpg -> paper
/Users/rodynaamr/.cache/kagglehub/datasets/mostafaabla/garbage-classification/versions/1/garbage_classification/paper/paper730.jpg -> paper
/Users/rodynaamr/.cache/kagglehub/datasets/mostafaabla/garbage-classification/versions/1/garbage_classification/paper/paper718.jpg -> paper


In [14]:
def predict_image_with_unknown(image_path, threshold=0.6):
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")

    img = Image.open(image_path).convert("RGB")

    # Extract features
    vec = img2vec.get_vec(img)
    vec = np.array(vec).reshape(1, -1)

    # Predict probabilities
    probs = svm_model.predict_proba(vec)[0]
    max_prob = probs.max()
    pred_idx = np.argmax(probs)

    if max_prob < threshold:
        pred_label = "unknown"
    else:
        pred_label = label_encoder.inverse_transform([pred_idx])[0]

    print(f"Prediction: {pred_label} | Confidence: {max_prob:.4f}")
    return pred_label


In [18]:
all_predictions = []
for path in image_paths:
    pred = predict_image_with_unknown(path, threshold=0.6)
    all_predictions.append(pred)

# Example statistics
from collections import Counter
print(Counter(all_predictions))


NameError: name 'Image' is not defined

In [19]:
from sklearn.metrics import classification_report, accuracy_score
from collections import Counter
import joblib

LABEL_ENCODER_PATH = "/Users/rodynaamr/Image_Classification_SVM_kNN/src/models/label_encoder.pkl"
label_encoder = joblib.load(LABEL_ENCODER_PATH)

# Map Kaggle labels to model classes + unknown
known_classes = list(label_encoder.classes_)
y_true = []
y_pred = []

for path, true_label in zip(image_paths, labels):
    # Map unseen labels to 'unknown'
    if true_label not in known_classes:
        mapped_label = "unknown"
    else:
        mapped_label = true_label
    y_true.append(mapped_label)

    # Predict
    pred_label = predict_image_with_unknown(path, threshold=0.6)
    y_pred.append(pred_label)

# Metrics
print("Accuracy:", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred))
print("Prediction counts:", Counter(y_pred))


NameError: name 'Image' is not defined